In [18]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [2]:
def preprocess_clinical_data(oppScrData, mean=True):
    clinical_data = oppScrData.filter(['Clinical F/U interval  [d from CT]','BMI','BMI >30', 'Sex', 'Tobacco', 'Met Sx', 'FRAX 10y Fx Prob (Orange-w/ DXA)',
                                'FRAX 10y Hip Fx Prob (Orange-w/ DXA)','FRS 10-year risk (%)', 'DEATH [d from CT]' , 'Age at CT'], axis=1)
    # Replace all _,X,blanks with nan
    clinical_data = clinical_data.replace(r'_', np.nan, regex=True)
    clinical_data = clinical_data.replace(r'X', np.nan, regex=True)
    clinical_data = clinical_data.replace(r'^\s*$', np.nan, regex=True)

    # Fill na in bmi column with mean
    clinical_data['BMI'].fillna(value=clinical_data['BMI'].mean(skipna=True), inplace=True)
    
    # Fill na in bmi>30 column based on bmi col
    clinical_data.loc[clinical_data.BMI>30, 'BMI >30'] = 1
    clinical_data.loc[clinical_data.BMI<=30, 'BMI >30'] = -1
    
    clinical_data['Sex'] = np.where(clinical_data['Sex']=='Male',1,-1)
    clinical_data['Met Sx'] = np.where(clinical_data['Met Sx']=='Y',1,-1) 

    # Treat no data in tobacco as no tobacco usage 
    clinical_data['Tobacco'] = np.where(clinical_data['Tobacco']=='Yes',1,-1) 
    
    #Fill death col
    #fill_death_col(clinical_data)

    clinical_data['FRS 10-year risk (%)'] = clinical_data['FRS 10-year risk (%)'].replace("<1", 0.01, regex=True)
    clinical_data['FRS 10-year risk (%)'] = clinical_data['FRS 10-year risk (%)'].replace(">30", 0.30, regex=True)
    clinical_data['FRS 10-year risk (%)'] =  clinical_data['FRS 10-year risk (%)'] * 100
 
    cols_to_be_filled = ['FRAX 10y Fx Prob (Orange-w/ DXA)','FRAX 10y Hip Fx Prob (Orange-w/ DXA)','FRS 10-year risk (%)']
    for c in cols_to_be_filled:
      if mean:  
        clinical_data[c].fillna(value=clinical_data[c].mean(skipna=True), inplace=True)
      else :
        clinical_data[c].fillna(value=clinical_data[c].median(skipna=True), inplace=True)

    return [clinical_data, np.array(clinical_data, dtype=np.float32)]

In [6]:
oppScrData = pd.read_excel (r'sample_data/OppScrData.xlsx')  

In [8]:
clinical_data, clinical_data_np = preprocess_clinical_data(oppScrData)

In [10]:
selected_rows = clinical_data[~clinical_data['DEATH [d from CT]'].isnull()]
selected_rows

selected_row2=clinical_data[clinical_data['DEATH [d from CT]'].isnull()]

X=selected_rows.iloc[:, 0:9]
X
X2=selected_row2.iloc[:, 0:9]

y=selected_rows.iloc[:, 9]
y
y2=selected_row2.iloc[:, 9]

from sklearn.preprocessing import MinMaxScaler,StandardScaler
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(X), columns = X.columns)
X2=pd.DataFrame(sc.fit_transform(X2), columns = X.columns)

In [11]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
RFparams2={'n_estimators':[50],
          'min_samples_split':[16],
          'min_samples_leaf': [4],
           'max_features': [6],
           'max_depth': [20],
           'bootstrap': [True]}

rf = RandomForestRegressor(random_state = 21)
clf = GridSearchCV(rf,RFparams2,refit=True)
clf.fit(X_train,y_train)      
clf.best_estimator_.score(X_test, y_test)

0.9675514805768979

In [13]:
p = clf.predict(X_test)
null = clf.predict(X2)
Mse = mean_squared_error(y_test, p)
error = mean_absolute_error(y_test, p)
mape = mean_absolute_percentage_error(y_test, p)
print("RMSE: %.2f" % (Mse**(1/2.0)))
print("MApe: %.2f" %(mape))
print("error",error)

RMSE: 237.25
MApe: 0.55
error 160.64980233315563


In [14]:
selected_row2['DEATH [d from CT]']=null

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
X3=pd.concat([selected_rows,selected_row2])

In [16]:
y3=X3.iloc[:,9]
X3=X3.iloc[:, 0:9]
X_train3, X_test3, y_train3, y_test3  = train_test_split(X3, y3, test_size=0.2, random_state=0)

In [19]:
knn2 = KNeighborsRegressor(n_neighbors=9)
weight_options = ["uniform", "distance"]
param_grid2 = dict(weights = weight_options)
    
grid = GridSearchCV(knn2, param_grid2)
grid.fit(X_train3,y_train3)
p3=grid.predict(X_test3)
Mse = mean_squared_error(y_test3, p3)

mape=mean_absolute_percentage_error(y_test3, p3)
print("RMSE: %.2f" % (Mse**(1/2.0)))
print("MApe: %.2f" %(mape))

mean_absolute_error(y_test3, p3)

RMSE: 194.44
MApe: 0.07


89.7835556471623